# [모듈 1.1] EventBridge를 통해 CodePipeline 수행

이 노트북은 CodeCommit 레파지토리에 훈련 파일(csv)을 push 하면, EventBridge를 통해 CodePipeline을 실행하는 노트북 입니다.

## 참고 자료
- Customizing triggers for AWS CodePipeline with AWS Lambda and Amazon CloudWatch Events 
   (https://aws.amazon.com/blogs/devops/adding-custom-logic-to-aws-codepipeline-with-aws-lambda-and-amazon-cloudwatch-events/)
- Boto3 Docs : EventBridge
   (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html#EventBridge.Client.put_events)



# 1. 환경 확인

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import sagemaker
import boto3
sys.path.append('./src')


# 2. 파라미터 세팅

In [2]:
# 기본 정보
region = sagemaker.Session().boto_region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
training_files = 'training'

# Codecommit Repo Name
repository_name = "ncf-train"

# code pipeline name, arn, role_arn
codepipeline_name = "ncf-training-code-pipeline"
codepipeline_arn = boto3.client('codepipeline').get_pipeline(name = codepipeline_name).get('metadata').get('pipelineArn')
codepipeline_role_arn = boto3.client('codepipeline').get_pipeline(name = codepipeline_name).get('pipeline').get('roleArn')

# Event Bus Name
eventbridge_eventbus_name = "mlopseventbus"

# EventBridge Rule Name
eventbridge_rule_name = "codepipelinerule"

# 3. EventBridge Rule 정의

In [3]:
# 기존 Target, Rule 삭제
#eventclient = boto3.client('events')
#eventresponse = eventclient.delete_rule(Name = eventbridge_rule_name)

## 3.1 event_bus 생성

In [4]:
eventclient = boto3.client('events')

eventresponse = eventclient.create_event_bus(
    Name = eventbridge_eventbus_name
)

print(eventresponse)

{'EventBusArn': 'arn:aws:events:us-east-1:376278017302:event-bus/mlopseventbus', 'ResponseMetadata': {'RequestId': '8cb2859a-55f5-4677-94d0-236d8b7a6122', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8cb2859a-55f5-4677-94d0-236d8b7a6122', 'content-type': 'application/x-amz-json-1.1', 'content-length': '79', 'date': 'Wed, 14 Dec 2022 16:29:41 GMT'}, 'RetryAttempts': 0}}


## 3.2 EventPattern 정의

#### 아래 CodeCommit arn 출력값을 복사해서 resources 에 붙여넣으세요

In [5]:
# Codecommit Arn
repository_arn = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('Arn')
print(repository_arn)

arn:aws:codecommit:us-east-1:376278017302:ncf-train


In [17]:
eventclient = boto3.client('events')

eventresponse = eventclient.put_rule(
    Name = eventbridge_rule_name,
    EventBusName = eventbridge_eventbus_name,
    EventPattern = """{
        "source": ["aws.codecommit"],
        "detail-type": ["CodeCommit Comment on Commit"],
        "resources": ["arn:aws:codecommit:us-east-1:376278017302:ncf-train"]
    }"""
)

print(eventresponse)

{'RuleArn': 'arn:aws:events:us-east-1:376278017302:rule/mlopseventbus/codepipelinerule', 'ResponseMetadata': {'RequestId': '631c50ca-be78-4fa7-af25-8aacf32fd7cf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '631c50ca-be78-4fa7-af25-8aacf32fd7cf', 'content-type': 'application/x-amz-json-1.1', 'content-length': '87', 'date': 'Wed, 14 Dec 2022 16:41:58 GMT'}, 'RetryAttempts': 0}}


## 3.2 Event Target 설정

In [11]:
eventresponse = eventclient.put_targets(
    Rule = eventbridge_rule_name,
    EventBusName = eventbridge_eventbus_name,
    Targets = [
        {
            'Arn' : codepipeline_arn,
            'Id' : codepipeline_name,
            'RoleArn' : codepipeline_role_arn
        }
    ]
)

print(eventresponse)

{'FailedEntryCount': 0, 'FailedEntries': [], 'ResponseMetadata': {'RequestId': 'b4b3ca20-e997-4e5a-bb7e-38f56102ce6c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b4b3ca20-e997-4e5a-bb7e-38f56102ce6c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '41', 'date': 'Wed, 14 Dec 2022 16:32:36 GMT'}, 'RetryAttempts': 0}}


# 4. 리파지토리 로컬에 클로닝
### Repository URL을 확인 후 복사해서 아래의 repository_url에 붙여넣기 해주세요.
### repository_url :
```
repository_url =  "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train"
```

In [9]:
repository_url =  "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train"
# repository_url = "<Repository URL>"

In [10]:
%%sh -s {repository_url} {repository_name}

repository_url=$1
repository_name=$2

rm -rf $repository_name

git clone $repository_url 

Cloning into 'ncf-train'...
remote: Counting objects: 65, done.        


# 6. 훈련 파일을 로컬 레파지토리 디렉토리에 복사

In [19]:
%%sh -s {repository_name} {training_files}
repository_name=$1
training_files=$2

cp $training_files/* $repository_name
ls $repository_name

codebuild-buildspec-221113.yml
codebuild-buildspec-origin.yml
codebuild-buildspec.yml
code_location.json
CONTRIBUTING.md
dummy.csv
img
LICENSE
origin-sagemaker-pipelines-project.ipynb
pipelines
README.md
sagemaker-pipelines-project.ipynb
setup.cfg
setup.py
tests
tox.ini


# 7. 훈련 파일을 리파지토리에 푸시

In [21]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
echo $PWD
git add .
git commit -m "Add new files"
git push origin master


/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/6_sm-eventbridge/ncf-train
[master 4ba03d2] Add new files
 Committer: EC2 Default User <ec2-user@ip-172-16-23-233.ec2.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 1 file changed, 3 deletions(-)
 delete mode 100644 dummy.csv


remote: Validating objects: 100%        
To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train
   fc9e92f..4ba03d2  master -> master
